In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle
import random
from pathlib import Path

import seaborn as sns
import pandas as pd
import torch
from torch.nn.functional import elu
import numpy as np
import matplotlib.pyplot as plt

from alpaca.uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from alpaca.analysis.metrics import uq_ll
from alpaca.model.ensemble import MLPEnsemble
from alpaca.uncertainty_estimator import build_estimator
from alpaca.analysis.metrics import get_uq_metrics

plt.rcParams['figure.facecolor'] = 'white'


In [ ]:
SEED = 10
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.cuda.set_device(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
folder = Path('./data/regression')
files = sorted([file for file in os.listdir(folder) if file.endswith('.pickle')])
# files = [file for file in files if file.startswith('kin8')]



In [ ]:
def load_setup(file):
    print(file)
    with open(folder / 'log_exp.log', 'w') as f:
        f.write(f'{cnt} / {len(files)}')
    with open(folder / file, 'rb') as f:
        dct = pickle.load(f)
    print(file)
    config = dct['config']
    config['n_ue_runs'] = 1
    config['acc_percentile'] = .1
    state_dict = dct['state_dict']
    x_train, y_train, x_val, y_val, x_scaler, y_scaler = dct['data']

    ensemble = MLPEnsemble(
        config['layers'], n_models=config['n_ens'], activation = elu,
        reduction='mean')
    ensemble.load_state_dict(state_dict)

    model = ensemble.models[2]
    return model, ensemble, x_train, y_train, x_val, y_val, x_scaler, y_scaler


In [ ]:
accumulate = []
data = []

for cnt, file in enumerate(files):
    model, ensemble, x_train, y_train, x_val, y_val, x_scaler, y_scaler = load_setup(file)
    estimator_ensemble = build_estimator('emcdue', ensemble, nn_runs=100)
    estimator_single = build_estimator('mcdue', model, nn_runs=100)

    x_val_tensor = torch.tensor(x_val)
    unscale = lambda y : y_scaler.inverse_transform(y)

    predictions = model(x_val_tensor.cuda()).cpu().detach().numpy()
    errors = predictions - y_val
    ue_single = estimator_single.estimate(torch.Tensor(x_val).double().cuda())
    ll_single = uq_ll(errors, ue_single)


    predictions = ensemble(x_val_tensor.cuda()).cpu().detach().numpy()
    errors = predictions - y_val
    ue_ensemble = estimator_ensemble.estimate(torch.Tensor(x_val).double().cuda())
    ll_ensemble = uq_ll(errors, ue_ensemble)

    accumulate.append([file[:4], 'single', ll_single])
    accumulate.append([file[:4], 'ensemble', ll_ensemble])
    #
    # print(file)
    # print('Single', rmse_single)
    # print('Ensemble', rmse_ensemble)



In [ ]:
df = pd.DataFrame(accumulate, columns=['dataset', 'type', 'll'])
sns.boxplot('dataset', 'll', hue='type', data=df)
plt.savefig('ll_emcdue.png', dpi=150)



